In [19]:
import pandas as pd
import string
import numpy as np

In [20]:
laptop_data = pd.read_csv('clean_data/clean_data.csv')

In [21]:

class DebugFeatureProcess:
    def __init__(self, features):
        self.features = features
    def printTokens(self, idx):
        feature = self.features[idx].replace('\xa0', ' ')
        feature = feature.replace('-', ' ')
        feature = feature.replace(',', ' ')
        tokens = [token.strip(string.punctuation).lower() for token in feature.split(' ')]
        tokens = [token for token in tokens if token != '']
        print(tokens)
    def printFeature(self, idx):
        print(self.features[idx])
    def printAll(self, indices):
        for idx in indices:
            print(str(idx), end = ": ")
            self.printTokens(idx)
            print(str(idx), end = ": ")
            self.printFeature(idx)

In [22]:
def isfloat(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [23]:
def memoryProcess(memories):
    anomalies = [1250, 3532, 5863]
    '''
    5863: 8 GB  ,, DDRL3
    3532: 2048 MB  ,, LPDDR3 MHz SDRAM, on-board 
    1250: 16 GB  ,, DDR4 2133/2400 MHz SO-DIMM
    '''
    index = memories.index
    num_of_ram_sticks = pd.Series(index = index, dtype = 'int32') 
    ram_capacity = pd.Series(index = index, dtype = 'float64') 
    bus = pd.Series(index = index, dtype = 'float64')
    ddr = pd.Series(index = index, dtype = 'int32')
    for idx, memory in zip(index, memories):
        try:
            if type(memory) != str:
                continue
            if idx in anomalies:
                continue
            memory = memory.lower()
            memory = memory.replace('\xa0', ' ')
            memory = memory.replace('-', ' ')
            memory = memory.replace(',', ' ')
            tokens = [token.strip(string.punctuation) for token in memory.split()]
            tokens = [token for token in tokens if token != '']
            for i in range(len(tokens)):

                if 'x' in tokens[i]:
                    num = tokens[i][:tokens[i].find('x')]
                    if num.isnumeric():
                        num_of_ram_sticks[idx] = int(num)

                if np.isnan(ram_capacity[idx]):
                    if tokens[i] == 'gb':
                        ram_capacity[idx] = round(float(tokens[i - 1]))
                    elif tokens[i] == 'mb':
                        ram_capacity[idx] = round(float(tokens[i - 1]) / 1024)

                if np.isnan(bus[idx]):    
                    if tokens[i] == 'mhz':
                        bus[idx] = float(tokens[i-1])
                    elif 'mhz' in tokens[i]:
                        num = tokens[i][:-3]
                        if isfloat(num):
                            bus[idx] = float(tokens[i][:-3])
                        else:
                            bus[idx] = float(tokens[i-1])

                if len(tokens[i]) > 2:
                    if tokens[i][:3] == "ddr":
                        if len(tokens[i]) > 3:
                            ddr[idx] = int(tokens[i][3])
                        else:
                            ddr[idx] = 1
        except Exception as e:
                print(idx)
                print(str(e))
    return pd.DataFrame({'num_of_ram_sticks' : num_of_ram_sticks,
            'ram_capacity_gb' : ram_capacity,
            'bus_mhz': bus,
            'ddr': ddr})

In [24]:
processed_memory = memoryProcess(laptop_data["Memory"])

902
could not convert string to float: '2133/2400'
4815
invalid literal for int() with base 10: 'l'


In [25]:
def storageProcess(storages):
    anomalies = [7619, 4204, 3705, 3407, 3357, 3109, 3082, 2701, 2429, 1972, 1909, 617, 987, 1255, 1868, 2539, 5270, 5990, 6073, 6499, 7291, 7575, 1826, 7326, 8739]
    '''
    8739:  Up to 2TB10 M.2 PCIe SSD
    7619:  512 GB Phison PS5016 ESR512GTLG-E6GBTNB4, 410 GB free
    4204:  512GGB SSD
    3705:  256GB SSD M.2 SATA6Gb/s
    3407: Samsung SSD PM830 256 GByte mSATA, 256 GB  
    3357: Samsung SSD PM830 256 GByte mSATA, 256 GB  
    3109:  256GB/512GB SATA3 SSD
    3082:  512 GB SATA 6.0Gb/s M.2 ssd
    2701:  1TB SATA HDD+128GB SSD
    2429:  512 GBM PCIe/NVMe (2280/M-Key Slot)
    1972:  256 GByte M.2-SSD + 1 TByte HDD
    1909:  2x512GB PCIE SSD
    617: Hitachi HTS545032A7E380 + 32 GByte Samsung SSD 830, 320 GB
    987: 2x ADATA XM11 256 GByte, 512 GB
    1255:  1 TByte HGST Travelstar 7K1000 (7200rpm, 32 MByte Cache) + 256 GByte Transcend MTS800 (M.2, SATA)
    1868:  3x512 GB NVMe SSD
    2539: Ramaxel RTNTE256PCA8EADL, M.2 GB
    5270:  1 TB HDD + 2x512 GB SSD
    5990: Ramaxel RTNTE256PCA8EADL, M.2 GB
    6073:  2x256 GB SSD + 1 TB HDD
    6499:  2x256 GB SSD + 1 TB HDD
    7291:  1TB GB SSD
    7575:  1 TB GB SSD PCIe NVMe M.2 Gen 4
    1826:  TB HDD + 128GB SSD
    7326:  TB SSD
    '''
    index = storages.index
    storage_capacity = pd.Series(index = index, dtype = 'float64') #gb
    rpm = pd.Series(index = index, dtype = 'float64') #rpm
    ssd = pd.Series(0, index = index, dtype = 'int32')
    hdd = pd.Series(0, index = index, dtype = 'int32')
    for idx, storage in zip(index, storages):
        try:
            if type(storage) != str:
                continue
            if idx in anomalies:
                continue
            storage = storage.lower()
            storage = storage.replace('\xa0', ' ')
            storage = storage.replace('-', ' ')
            storage = storage.replace(',', ' ')
            tokens = [token.strip(string.punctuation) for token in storage.split()]
            tokens = [token for token in tokens if token != '']
            
            capacity = 0
            
            for i in range(len(tokens)):
                
                if tokens[i] == 'gb':
                    capacity += float(tokens[i-1])
                elif 'gb' in tokens[i]:
                    if isfloat(tokens[i][:-2]):
                        capacity += float(tokens[i][:-2])
                
                if tokens[i] == 'tb':
                    capacity += float(tokens[i-1]) * 1024
                elif 'tb' in tokens[i]:
                    if isfloat(tokens[i][:-2]):
                        capacity += float(tokens[i][:-2])
                    
                if tokens[i] == 'rpm':
                    rpm[idx] = float(tokens[i-1])
                    hdd[idx] = 1
            
            if capacity != 0:
                storage_capacity[idx] = capacity
                
            if 'hdd' in storage:
                hdd[idx] = 1
            
            if 'ssd' in storage:
                ssd[idx] = 1
                    
        except Exception as e:
            DebugFeatureProcess(storages).printAll([idx])
            print(str(e))
            print()
    return pd.DataFrame({'storage_capacity_gb':storage_capacity,
                         'rpm': rpm,
                         'hdd': hdd,
                         'ssd': ssd})

In [26]:
processed_storage = storageProcess(laptop_data["Storage"])

1357: ['tb', 'hdd', '128gb', 'ssd']
1357:  TB HDD + 128GB SSD
could not convert string to float: 'ssd'

1398: ['3x512', 'gb', 'nvme', 'ssd']
1398:  3x512 GB NVMe SSD
could not convert string to float: '3x512'

1970: ['ramaxel', 'rtnte256pca8eadl', 'm.2', 'gb']
1970: Ramaxel RTNTE256PCA8EADL, M.2 GB  
could not convert string to float: 'm.2'

4299: ['1', 'tb', 'hdd', '2x512', 'gb', 'ssd']
4299:  1 TB HDD + 2x512 GB SSD
could not convert string to float: '2x512'

4928: ['ramaxel', 'rtnte256pca8eadl', 'm.2', 'gb']
4928: Ramaxel RTNTE256PCA8EADL, M.2 GB  
could not convert string to float: 'm.2'

5008: ['2x256', 'gb', 'ssd', '1', 'tb', 'hdd']
5008:  2x256 GB SSD + 1 TB HDD
could not convert string to float: '2x256'

5410: ['2x256', 'gb', 'ssd', '1', 'tb', 'hdd']
5410:  2x256 GB SSD + 1 TB HDD
could not convert string to float: '2x256'

6134: ['1tb', 'gb', 'ssd']
6134:  1TB GB SSD
could not convert string to float: '1tb'

6169: ['tb', 'ssd']
6169:  TB SSD
could not convert string to float: 

In [27]:
'''
    1868:  3x512 GB NVMe SSD
    5270:  1 TB HDD + 2x512 GB SSD
    6073:  2x256 GB SSD + 1 TB HDD
    6499:  2x256 GB SSD + 1 TB HDD
'''
processed_storage.loc[1868] = [3 * 512, np.nan, 0, 1]
processed_storage.loc[5270] = [1024 + 2 * 512, np.nan, 1, 1]
processed_storage.loc[6073] = [2*256 + 1024, np.nan, 1, 1]
processed_storage.loc[6499] = [2*256 + 1024, np.nan, 1, 1]

In [28]:
'''
    8739:  Up to 2TB10 M.2 PCIe SSD
    7800: Phison PS5013 311CD0512GB, 512 GB  
    7799: Phison PS5013 311CD0512GB, 512 GB  
    4204:  512GGB SSD
    3705:  256GB SSD M.2 SATA6Gb/s
    3407: Samsung SSD PM830 256 GByte mSATA, 256 GB  
    3357: Samsung SSD PM830 256 GByte mSATA, 256 GB  
    3082:  512 GB SATA 6.0Gb/s M.2 ssd
    2701:  1TB SATA HDD+128GB SSD
    2429:  512 GBM PCIe/NVMe (2280/M-Key Slot)
    1972:  256 GByte M.2-SSD + 1 TByte HDD
    1909:  2x512GB PCIE SSD
    617: Hitachi HTS545032A7E380 + 32 GByte Samsung SSD 830, 320 GB
    987: 2x ADATA XM11 256 GByte, 512 GB
    1255:  1 TByte HGST Travelstar 7K1000 (7200rpm, 32 MByte Cache) + 256 GByte Transcend MTS800 (M.2, SATA)
'''
processed_storage.loc[8739] = [1024 * 2, np.nan, 0, 1]
processed_storage.loc[7800] = [512, np.nan, 0, 0]
processed_storage.loc[7799] = [512, np.nan, 0, 0]
processed_storage.loc[4204] = [512, np.nan, 0, 1]
processed_storage.loc[3705] = [256, np.nan, 0, 1]
processed_storage.loc[3407] = [256, np.nan, 0, 1]
processed_storage.loc[3357] = [256, np.nan, 0, 1]
processed_storage.loc[3082] = [512, np.nan, 0, 1]
processed_storage.loc[2701] = [1024 + 128, np.nan, 1, 1]
processed_storage.loc[2429] = [512, np.nan, 0, 0]
processed_storage.loc[1972] = [256 + 1024, np.nan, 1, 1]
processed_storage.loc[1909] = [2 * 512, np.nan, 0, 1]
processed_storage.loc[617] = [320, np.nan, 0, 1]
processed_storage.loc[987] = [512, np.nan, 0, 0]
processed_storage.loc[1255] = [1024 + 256, 7200, 0, 0]

In [29]:
def displayProcess(displays):
    anomalies = [1245, 2077, 3590]
    '''
    1245: 4 USB 3.0 / 3.1 Gen1, 1 USB 3.1 Gen2, 1 Thunderbolt, 1 HDMI, 1 DisplayPort, 
    1 Infrared, Audio Connections: 3.5 mm earphones, Toslink, Card Reader: SDXC w. UHS-I support
    2077:  16:9, 1920 x 1080 pixel, IPS, glossy: no
    3590: 4 USB 3.0 / 3.1 Gen1, 1 USB 3.1 Gen2, 1 Thunderbolt, 1 HDMI, 1 DisplayPort,
    1 Kensington Lock, Audio Connections: 3.5 mm audio jack, Card Reader: SD
    '''
    index = displays.index
    inch = pd.Series(index = index, dtype = 'float64') 
    pixel_height = pd.Series(index = index, dtype = 'int32')
    pixel_width = pd.Series(index = index, dtype = 'int32')
    for idx, display in zip(index, displays):
        if idx in anomalies:
            continue
        if type(display) != str:
            continue
        try:
            display = display.lower()
            tokens = [token.strip(string.punctuation + ' ') for token in display.split(',')]
            tokens = [token for token in tokens if token != '']
            inch_part, pixel_part = tokens[0], tokens[1]
            inch[idx] = float(inch_part.split()[0])
            tokens = pixel_part.split()
            pixel_height[idx], _, pixel_width[idx] = int(tokens[0]), tokens[1], int(tokens[2])
        except Exception as e:
            print(str(idx), end = ": ")
            DebugFeatureProcess(displays).printFeature(idx)
            print(str(e))
    return pd.DataFrame({'inch': inch,
                         'pixel_height': pixel_height,
                         'pixel_width': pixel_width})

In [30]:
processed_display = displayProcess(laptop_data["Display"])

1597:  16:9, 1920 x 1080 pixel, IPS, glossy: no
could not convert string to float: '16:9'
2736: 4 USB 3.0 / 3.1 Gen1, 1 USB 3.1 Gen2, 1 Thunderbolt, 1 HDMI, 1 DisplayPort, 1 Kensington Lock, Audio Connections: 3.5 mm audio jack, Card Reader: SD
invalid literal for int() with base 10: '3.1'


In [31]:
# 2077:  16:9, 1920 x 1080 pixel, IPS, glossy: no
processed_display.loc[2077] = [np.nan, 1920, 1080]

In [32]:
laptop_data["Memory"]

0                16 GB  ,, 4x 4096 MB DDR3-RAM (1333 MHz)
1                          4 GB  ,, A-Data AD73I1C1674EV 
2       8 GB  ,, 2x 4096 MB DDR3-RAM (1333 MHz), max. ...
3                                                  4 GB  
4                               1024 MB  ,, DDR2, max 2GB
                              ...                        
7682                                       16 GB  ,, DDR4
7683                                        8 GB  ,, DDR4
7684    8 GB  ,, DDR4-3200, Single-Channel-Mode, one m...
7685                                  16 GB  ,, DDR4-3200
7686                   8 GB  ,, DDR4 SDRAM single-channel
Name: Memory, Length: 7687, dtype: object

In [33]:
processed_memory

,num_of_ram_sticks,ram_capacity_gb,bus_mhz,ddr
0,4.0,16.0,1333.0,3.0
1,NaN,4.0,NaN,NaN
2,2.0,8.0,1333.0,3.0
3,NaN,4.0,NaN,NaN
4,NaN,1.0,NaN,2.0
...,...,...,...,...
7682,NaN,16.0,NaN,4.0
7683,NaN,8.0,NaN,4.0
7684,NaN,8.0,NaN,4.0
7685,NaN,16.0,NaN,4.0


In [34]:
laptop_data["Storage"]

0       2x WDC Scorpio Blue WD5000BEVT-22A0RT0 (RAID 0...
1                           Toshiba MK5059GSXP , 500 GB  
2         WDC Scorpio Black WD7500BPKT-75PKA4T0, 750 GB  
3                                                     NaN
4                          Fujitsu MJA2250BH G2, 250 GB  
                              ...                        
7682                                              1TB SSD
7683                                            512GB SSD
7684                                           , 256 GB  
7685                      1 TB PCIe 3x4 NVMe M.2 2280 SSD
7686                                           , 512 GB  
Name: Storage, Length: 7687, dtype: object

In [35]:
laptop_data["Display"]

0       17.30 inch 16:9, 1920 x 1080 pixel, CMO1720, F...
1       14.00 inch 16:9, 1366 x 768 pixel, LG LP140WH2...
2       15.60 inch 16:9, 1920 x 1080 pixel, LG Philips...
3          13.30 inch 16:9, 1366 x 768 pixel, glossy: yes
4       10.10 inch 16:9, 1024 x 600 pixel, WSVGA LED T...
                              ...                        
7682    15.60 inch 16:9, 1920 x 1080 pixel 141 PPI, IP...
7683    15.60 inch 16:9, 1920 x 1080 pixel 141 PPI, IP...
7684    15.60 inch 16:9, 1920 x 1080 pixel 141 PPI, AU...
7685           14.00 inch 16:9, 1920 x 1080 pixel 157 PPI
7686    15.60 inch 16:9, 1920 x 1080 pixel, CEC PANDA ...
Name: Display, Length: 7687, dtype: object

In [36]:
processed_display

,inch,pixel_height,pixel_width
0,17.3,1920.0,1080.0
1,14.0,1366.0,768.0
2,15.6,1920.0,1080.0
3,13.3,1366.0,768.0
4,10.1,1024.0,600.0
...,...,...,...
7682,15.6,1920.0,1080.0
7683,15.6,1920.0,1080.0
7684,15.6,1920.0,1080.0
7685,14.0,1920.0,1080.0
